In [ ]:
import cv2
import numpy as np
import matplotlib as plt
import pandas as pd
import time
import tqdm
import os
import matplotlib.pyplot as plt
from scipy.signal import chirp, find_peaks, peak_widths
import glob
class call_num():
    def __init__(self):
        self.number = 0
    def add(self):
        self.number = self.number + 1
    def print(self):
        return self.number

def start_pint(x):
    '''input: Frame list(brightness)'''
    peaks, _ = find_peaks(x, height=-40)
    results_half = peak_widths(x, peaks, rel_height=0.5)
    result = peaks-results_half[0]/2
    fig, ax = plt.subplots()
    ax.plot(x)
    ax.plot(peaks, x[peaks], 'x')
    plt.hlines(*results_half[1:], color="C2")
    plt.vlines(result,0,1,color='red')
    plt.savefig('./{}.png'.format(call_num.print()))
    plt.show()
    plt.close()
    return result, results_half[0]-results_half[0]%11

def video_to_frames(input_loc):
    time_start = time.time()
    cap = cv2.VideoCapture(input_loc)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    image_max = []
    image_value=np.array([],dtype=int)
    print ("Converting video..\n")
    while cap.isOpened():
        ret, frame = cap.read()
        if np.max(frame)>25 :
            image_value = np.append(image_value, 1)
        else :
            image_value = np.append(image_value, 0)
        image_max.append(np.max(frame))
        count = count + 1
        if (count > (video_length-1)):
            time_end = time.time()
            cap.release()
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds forconversion." % (time_end-time_start))
            break
    fig, ax = plt.subplots()
    ax.plot(image_max)
    plt.savefig('./{}_1.png'.format(call_num.print()))
    plt.show()

    plt.close()
    return np.vstack((np.trunc(start_pint(image_value)[0]).astype('int'), np.round(start_pint(image_value)[1]/11).astype('int'))).T ##start point, interval

def make_dataset(input_loc, output_loc, start_point, number=11):
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    cap = cv2.VideoCapture(input_loc)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1

    count = 0
    all_point = []
    dataset = []

    for i, jump in start_point:
        if jump <2:
            pass
        else:
            for _ in range(number):
                all_point.append(i+jump)
                i+=jump
            print(i, jump)
    while cap.isOpened():
        ret, frame = cap.read()
        if count in all_point:
            dataset.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        else:
            pass
        count = count + 1
        if (count > (video_length-1)):
            cap.release()
            break

    return dataset, number


def stack_array(data, number,output_loc, name):
    cycle = 0
    for i in range(int(np.shape(data)[0]/number)):
        j = number * i
        """If data number change stack change"""
        result = np.stack([data[0+j], data[1+j], data[2+j], data[3+j], data[4+j],
                           data[5+j], data[6+j], data[7+j], data[8+j], data[9+j]], axis = 2)

        result_copy = croping(result)
        np.save(output_loc+name+'_{}'.format(call_num.print()),result_copy)
        print(result_copy.shape, call_num.print(), np.max(result_copy[:,:,9]))

        fig, ax = plt.subplots(2,5)
        for i in range(2):
            for j in range(5):
                ax[i][j].imshow(result_copy[:,:,i*5+j])
                ax[i][j].axis('off')
        plt.savefig('./test_figure/{}/{}.png'.format(name,call_num.print()))
        plt.close()
        call_num.add()
        cycle = cycle + 1
        if cycle == 20:
            break
    return


def croping(frame):
    grayFrame = frame[350:1050, 320:1020]
    grayFrame = cv2.resize(grayFrame, dsize=(210,210))
    return grayFrame




_input_loc = r'./raw_data/'
call_num = call_num()


list = ['around', 'b', 'fracture', 'ground', 'mechanics',
        'science', 'smile', 'strain', 'stress', 't']

num =0

input_loc = glob.glob(_input_loc+'{}*.mp4'.format(list[num]))[0]
name = os.path.basename(input_loc).split('_')[0]
output_loc = (r'./test_data/'+name+'/')
array = video_to_frames(input_loc)
dataset = make_dataset(input_loc, output_loc, array)
stack_array(dataset[0], dataset[1], output_loc, name)
print(input_loc, output_loc)
